
# FarmTech Solutions — Fase 5 (Entrega 1: Machine Learning)
**Aluno:** _substitua pelo seu nome_  
**Disciplina:** IA Aplicada ao Agronegócio  
**Objetivo:** Prever **Rendimento (t/ha)** e explorar **tendências de produtividade** via **clusterização**, com **boas práticas de ML**.

---

### O que este notebook faz
1. **Carrega os dados** `crop_yield.csv` (ou gera um dataset sintético compatível, caso o arquivo não esteja disponível).
2. **EDA:** estatísticas, checagens, correlações e gráficos.
3. **Tendências & Outliers:** KMeans, DBSCAN, PCA e detecção de outliers (IQR e IsolationForest).
4. **Modelagem (Regressão):** 5 algoritmos diferentes, com `Pipeline`, `ColumnTransformer`, validação, ajuste e **comparação de métricas**.
5. **Seleção & Exportação:** escolhe o melhor modelo e salva em `models/best_model.pkl`. Exporta métricas e predições.
6. **Reprodutibilidade:** semente aleatória, funções utilitárias e logs básicos.


In [ ]:

# Imports essenciais
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

# Pré-processamento e modelagem
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelos de Regressão
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# Validação e Métricas
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Clusterização e Redução de Dimensionalidade
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA

# Util
import joblib
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

BASE_DIR = Path('/mnt/data/FarmTech_Fase5_Entrega1')
DATA_DIR = BASE_DIR / 'data'
MODELS_DIR = BASE_DIR / 'models'
REPORTS_DIR = BASE_DIR / 'reports'
ARTIFACTS_DIR = BASE_DIR / 'artifacts'
DATA_PATH = DATA_DIR / 'crop_yield.csv'  # esperado no portal
SYNTH_PATH = DATA_DIR / 'crop_yield_sample.csv'



## 1. Carregamento dos dados
Tente carregar `data/crop_yield.csv`. Se não existir, geramos um dataset **sintético** com o mesmo esquema para facilitar a execução e a validação do pipeline.


In [ ]:

def generate_synthetic_dataset(n=600, n_culturas=5, random_state=RANDOM_STATE):
    rng = np.random.default_rng(random_state)
    culturas = [f'Cultura_{i+1}' for i in range(n_culturas)]
    cultura = rng.choice(culturas, size=n, replace=True)

    # Variáveis climáticas/solo (valores plausíveis, mas sintéticos)
    precipitacao = rng.normal(4.0, 2.0, size=n).clip(0, 20)  # mm/dia
    umid_espec = rng.normal(7.0, 1.8, size=n).clip(2, 15)    # g/kg
    umid_rel = rng.normal(65, 15, size=n).clip(10, 100)      # %
    temp2m = rng.normal(22, 6, size=n).clip(-2, 42)          # ºC

    # Relação não-linear com ruído + efeito por cultura
    base_yield = (
        0.8*precipitacao
        + 0.3*umid_espec
        - 0.15*np.maximum(temp2m-28, 0)**2 / 10  # penalidade calor extremo
        + 0.02*umid_rel
    )
    efeitos_cultura = {c: rng.normal(3.0 + i*0.5, 0.4) for i, c in enumerate(culturas)}
    rendimento = np.array([base_yield[i] + efeitos_cultura[cultura[i]] + rng.normal(0, 0.8) for i in range(n)])
    rendimento = rendimento.clip(0.5, None)  # t/ha

    df = pd.DataFrame({
        'Cultura': cultura,
        'Precipitação (mm dia 1)': precipitacao.round(2),
        'Umidade específica a 2 metros (g/kg)': umid_espec.round(2),
        'Umidade relativa a 2 metros (%)': umid_rel.round(1),
        'Temperatura a 2 metros (ºC)': temp2m.round(1),
        'Rendimento': rendimento.round(2)
    })
    return df

# Carregar ou gerar
if DATA_PATH.exists():
    df = pd.read_csv(DATA_PATH)
    origem = 'Arquivo fornecido (crop_yield.csv)'
else:
    df = generate_synthetic_dataset()
    df.to_csv(SYNTH_PATH, index=False)
    origem = 'Dataset sintético gerado (crop_yield_sample.csv)'
print('Origem dos dados:', origem)
df.head()



## 2. EDA — Análise Exploratória de Dados
Verificações essenciais: dimensões, tipos, valores ausentes, estatísticas descritivas e correlações.


In [ ]:

print('Formato:', df.shape)
print('\nTipos:\n', df.dtypes)
print('\nValores ausentes por coluna:\n', df.isna().sum())

display_cols = [c for c in df.columns if c != 'Cultura']
display(df.describe(include='all'))

# Distribuições (histogramas) — sem seaborn, um gráfico por chamada
for col in display_cols:
    plt.figure()
    df[col].hist(bins=30)
    plt.title(f'Distribuição de {col}')
    plt.xlabel(col)
    plt.ylabel('Frequência')
    plt.show()

# Correlação numérica
num_cols = [c for c in df.columns if df[c].dtype != 'O' and c != 'Rendimento'] + ['Rendimento']
corr = df[num_cols].corr()
plt.figure()
plt.imshow(corr, interpolation='nearest')
plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha='right')
plt.yticks(range(len(num_cols)), num_cols)
plt.title('Matriz de Correlação (numéricas)')
plt.colorbar()
plt.tight_layout()
plt.show()
corr['Rendimento'].sort_values(ascending=False)



## 3. Tendências de Produtividade e Outliers (Sem Supervisão)
Aplicamos **PCA** para 2D, **KMeans** (k=3..6) e **DBSCAN** para ver padrões de produtividade. Outliers via **IQR** e **IsolationForest**.


In [ ]:

# Selecionar features numéricas para clusterização
features = ['Precipitação (mm dia 1)', 'Umidade específica a 2 metros (g/kg)',
            'Umidade relativa a 2 metros (%)', 'Temperatura a 2 metros (ºC)', 'Rendimento']
X_unsup = df[features].copy()

# Padronizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_unsup)

# PCA 2D
pca = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)
print('Variância explicada (2D):', pca.explained_variance_ratio_.sum())

plt.figure()
plt.scatter(X_pca[:,0], X_pca[:,1], s=12)
plt.title('PCA (2D) — Dados padronizados')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.show()

# KMeans para vários k e inércia (elbow)
inertias = []
k_range = range(3, 7)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    km.fit(X_scaled)
    inertias.append(km.inertia_)

plt.figure()
plt.plot(list(k_range), inertias, marker='o')
plt.title('KMeans: Curva do Cotovelo (Inertia)')
plt.xlabel('k'); plt.ylabel('Inertia')
plt.show()

# Escolher k=4 (ajuste prático); você pode variar
k_best = 4
km = KMeans(n_clusters=k_best, random_state=RANDOM_STATE, n_init=10)
labels_km = km.fit_predict(X_scaled)

plt.figure()
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels_km, s=14)
plt.title(f'KMeans (k={k_best}) no espaço PCA 2D')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.show()

# DBSCAN
db = DBSCAN(eps=1.0, min_samples=10)
labels_db = db.fit_predict(X_scaled)

plt.figure()
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels_db, s=14)
plt.title('DBSCAN no espaço PCA 2D')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.show()

# Outliers por IQR em Rendimento
Q1 = df['Rendimento'].quantile(0.25)
Q3 = df['Rendimento'].quantile(0.75)
IQR = Q3 - Q1
lim_inf, lim_sup = Q1 - 1.5*IQR, Q3 + 1.5*IQR
mask_iqr = (df['Rendimento'] < lim_inf) | (df['Rendimento'] > lim_sup)
print('Outliers (IQR) em Rendimento:', mask_iqr.sum())

plt.figure()
plt.boxplot(df['Rendimento'])
plt.title('Boxplot de Rendimento (IQR)')
plt.ylabel('t/ha')
plt.show()

# IsolationForest em features (sem Cultura)
iso = IsolationForest(random_state=RANDOM_STATE, contamination=0.05)
out_iso = iso.fit_predict(X_unsup.drop(columns=['Rendimento']))
mask_iso = out_iso == -1
print('Outliers (IsolationForest):', mask_iso.sum())



## 4. Preparação para Modelagem Supervisionada (Regressão)
Transformação de **Cultura** por One-Hot e padronização de numéricas.


In [ ]:

target = 'Rendimento'
features_sup = ['Cultura', 'Precipitação (mm dia 1)',
                'Umidade específica a 2 metros (g/kg)',
                'Umidade relativa a 2 metros (%)',
                'Temperatura a 2 metros (ºC)']

X = df[features_sup].copy()
y = df[target].values

# Colunas
cat_cols = ['Cultura']
num_cols = [c for c in X.columns if c not in cat_cols]

# Transformadores
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

len(X_train), len(X_test)



## 5. Modelagem: 5 Algoritmos de Regressão
Avaliamos com **MAE**, **RMSE** e **R²**, usando `KFold` e depois **avaliamos no hold-out** (teste).


In [ ]:

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, random_state=RANDOM_STATE),
    'RandomForest': RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE),
    'GradientBoosting': GradientBoostingRegressor(random_state=RANDOM_STATE),
    'KNN': KNeighborsRegressor(n_neighbors=7),
    # Extra: 'SVR': SVR(), 'ElasticNet': ElasticNet(random_state=RANDOM_STATE)
}

results = []
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

for name, model in models.items():
    pipe = Pipeline(steps=[('prep', preprocessor), ('model', model)])
    cv_mae = -cross_val_score(pipe, X, y, cv=kf, scoring='neg_mean_absolute_error')
    cv_rmse = np.sqrt(-cross_val_score(pipe, X, y, cv=kf, scoring='neg_mean_squared_error'))
    cv_r2 = cross_val_score(pipe, X, y, cv=kf, scoring='r2')

    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    row = {
        'modelo': name,
        'cv_mae_mean': cv_mae.mean(),
        'cv_rmse_mean': cv_rmse.mean(),
        'cv_r2_mean': cv_r2.mean(),
        'test_mae': mean_absolute_error(y_test, preds),
        'test_rmse': rmse(y_test, preds),
        'test_r2': r2_score(y_test, preds)
    }
    results.append(row)

metrics_df = pd.DataFrame(results).sort_values('test_rmse')
metrics_path = ARTIFACTS_DIR / 'metrics.csv'
metrics_df.to_csv(metrics_path, index=False)
metrics_df



## 6. Seleção do Melhor Modelo e Exportação
Selecionamos o **menor RMSE de teste**. Salvamos o `Pipeline` completo em `models/best_model.pkl`.


In [ ]:

best_name = metrics_df.iloc[0]['modelo']
best_model = models[best_name]
best_pipe = Pipeline(steps=[('prep', preprocessor), ('model', best_model)])
best_pipe.fit(X, y)

best_model_path = MODELS_DIR / 'best_model.pkl'
joblib.dump(best_pipe, best_model_path)

print('Melhor modelo:', best_name)
print('Salvo em:', best_model_path)

# Predições no conjunto completo para auditoria
preds_all = best_pipe.predict(X)
audit_df = df.copy()
audit_df['Predito (t/ha)'] = preds_all
audit_path = ARTIFACTS_DIR / 'predicoes_completas.csv'
audit_df.to_csv(audit_path, index=False)
audit_df.head()



## 7. Interpretabilidade (Importâncias de Atributos — quando aplicável)
Para modelos baseados em árvore, exibimos importâncias aproximadas para **features numéricas**. (As dummies de `Cultura` são agregadas por média, apenas para ilustração simples.)


In [ ]:

def plot_feature_importances_from_pipeline(pipe, feature_names):
    model = pipe.named_steps['model']
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        plt.figure()
        plt.bar(range(len(importances)), importances)
        plt.title('Importâncias do Modelo (índices do espaço transformado)')
        plt.xlabel('Índice de feature transformada')
        plt.ylabel('Importância')
        plt.show()
    else:
        print('O modelo não expõe feature_importances_.')

# Treinar um pipe específico com RandomForest para visualizar importâncias
rf_pipe = Pipeline(steps=[('prep', preprocessor), ('model', RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE))])
rf_pipe.fit(X, y)
plot_feature_importances_from_pipeline(rf_pipe, features_sup)



## 8. Conclusões e Próximos Passos
- **Tendências:** clusters revelam grupos com diferentes perfis de clima/solo e rendimento.
- **Outliers:** IQR e IsolationForest ajudam a sinalizar cenários atípicos para inspeção.
- **Modelos:** comparamos **5 algoritmos**; o melhor foi salvo para uso futuro.
- **Próximos passos (sugestões):**
  - Experimentar **otimização de hiperparâmetros** (`GridSearchCV`/`RandomizedSearchCV`).
  - Avaliar **MAPE** e faixas de confiança.
  - Incluir variáveis adicionais (ex.: fertilização, variedade, pragas).
  - Produzir uma API simples (FastAPI) para servir o modelo (link com **Entrega 2: Nuvem**).
